In [19]:
import pickle
import numpy as np
import pandas as pd
from sklearn import base, feature_extraction, ensemble, model_selection, pipeline, compose, preprocessing, metrics
from sklearn.experimental import enable_halving_search_cv
import tensorflow as tf
from embedding_transformer import Doc2VecTransformer
from scikeras.wrappers import KerasClassifier
import optuna
import plotly
import pprint
import nbformat
from keyword_transformer import KeywordTransformer
from location_transformer import LocationTransformer
from text_stats_transformer import TextStatsTransformer

SCRIPT_NAME='DL-08'

In [2]:
# df_dataset = pd.read_csv('./train.csv', index_col='id')
# df_dataset.shape
# df_train, df_dev = model_selection.train_test_split(df_dataset, test_size=0.1)
# df_train.shape, df_dev.shape
# df_train.to_csv('./train-dataset.csv', index=True)
# df_dev.to_csv('./dev-dataset.csv', index=True)

In [3]:
df_train = pd.read_csv('./train-dataset.csv', index_col='id')
df_dev = pd.read_csv('./dev-dataset.csv', index_col='id')
df_train.shape, df_dev.shape

((6851, 4), (762, 4))

In [4]:
X_train = df_train.drop('target', axis=1)
Y_train = df_train['target']
X_dev = df_dev.drop('target', axis=1)
Y_dev = df_dev['target']
X_train.shape, Y_train.shape, X_dev.shape, Y_dev.shape

((6851, 3), (6851,), (762, 3), (762,))

In [5]:
class ArrayToDataFrameTransformer(base.BaseEstimator, base.TransformerMixin):
    def __init__(self, column_names_func):
        self.column_names_func = column_names_func
    
    def fit(self, X, y=None):
        return self  # Nothing to do
    
    def transform(self, X):
        return pd.DataFrame(X, columns=self.column_names_func())

In [6]:
enricher = compose.ColumnTransformer(transformers=[
    ('keywords', KeywordTransformer(keyword_file_name='./keywords_stats.csv'), ['keyword']),
    # ('location', LocationTransformer(), ['location']),
    ('text-features', TextStatsTransformer(), ['text'])
], remainder='passthrough', verbose_feature_names_out=False, verbose=True)

text_vec = feature_extraction.text.TfidfVectorizer(max_features=2000)

categorical_features = [
    'country',
    'state',
]
numerical_features = [
    'text_length', 
    'positive_factor',
    'token_sentiment_2'
]

column_transformer = compose.ColumnTransformer(transformers=[
    ('text_vec', text_vec, 'clean_text'),
    # ('one_hot', preprocessing.OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features),
    ('numerical', preprocessing.StandardScaler(), numerical_features)
], remainder='drop')

transformer = pipeline.Pipeline([
    ('enricher', enricher),
    ('pandas_mapper', ArrayToDataFrameTransformer(column_names_func=lambda : enricher.get_feature_names_out())),
    ('columns', column_transformer)
])


In [7]:
X_train_scaled = transformer.fit_transform(X_train).todense()
X_dev_scaled = transformer.transform(X_dev).todense()
X_train_scaled.shape, X_dev_scaled.shape

[ColumnTransformer] ...... (1 of 3) Processing keywords, total=   0.0s
[ColumnTransformer] . (2 of 3) Processing text-features, total=   3.0s
[ColumnTransformer] ..... (3 of 3) Processing remainder, total=   0.0s


((6851, 2003), (762, 2003))

In [8]:
INPUT_SIZE = X_train_scaled.shape[1]
OUTPUT_SIZE = 1
NN_SHAPE = [INPUT_SIZE, 32, 32, 16, 1]
BATCH_SIZE= X_train_scaled.shape[0]
MAX_EPOCHS = 40


def sigmoid(x):
    return tf.cast(tf.greater(tf.nn.sigmoid(x), .5), tf.int32)


def build_model(layer_dims=NN_SHAPE, use_dropout=False, dropout_rate_1=0.3, dropout_rate_2=0.1,
                learning_rate=1e-3, 
                use_emma=False, emma_momentum=0.99, 
                use_regularizer=False, regularizer=0.01,
                initializer='glorot_normal',
                activation='relu'
                ):
    n_layers = len(layer_dims)
    layers = []

    for l in range(1, n_layers-1):
        layer_kws = {}

        if use_regularizer:
            layer_kws['kernel_regularizer'] = tf.keras.regularizers.l2(regularizer)
        if initializer:
            layer_kws['kernel_initializer'] = initializer
        
        hidden_layer= tf.keras.layers.Dense(layer_dims[l], input_shape=(layer_dims[l-1],), activation=activation, **layer_kws)
        
        layers.append(hidden_layer)
        
        if use_dropout:
            if l==1:
                rate = dropout_rate_1
            elif l==2 and n_layers > 3:
                rate = dropout_rate_2
            else:
                rate = 0.0
            if rate > .0:
                layers.append(tf.keras.layers.Dropout(rate=rate))
    layers.append(tf.keras.layers.Dense(layer_dims[n_layers-1], activation='linear'))
    model = tf.keras.Sequential(layers)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, use_ema=use_emma, ema_momentum=emma_momentum),                 
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics='accuracy')
    return model


In [9]:
def train(X_train, Y_train, X_dev, Y_dev, model, batch_size=512):
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)
    
    model.fit(X_train, Y_train, batch_size=batch_size, epochs=MAX_EPOCHS, 
            validation_data=(X_dev, Y_dev),
            callbacks=[early_stopping],
            verbose=0)

    Y_predict = sigmoid(model.predict(X_dev))
    f1_score = metrics.f1_score(Y_dev, Y_predict)
    print(f"Validation F1 score: {f1_score}")

    return f1_score

In [10]:
model = build_model(use_regularizer=True, regularizer=1e-4)

In [11]:
train(X_train_scaled, Y_train, X_dev_scaled, Y_dev, model)

24/24 [==============================] - 0s 629us/step
Validation F1 score: 0.7515527950310559


0.7515527950310559

In [12]:
def create_objective_1(layer_dims):
    def objective1(trial):
        learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)
        batch_size = trial.suggest_categorical('batch_size', [BATCH_SIZE, BATCH_SIZE//2 + 1, BATCH_SIZE//4 + 1])
        regularizer = trial.suggest_float('regularizer', 1e-6, 1e-2, log=True)
        initializer = trial.suggest_categorical('initializer', ['glorot_normal', 'he_normal'])
        activation = trial.suggest_categorical('activation', ['relu', 'gelu', 'leaky_relu'])

        model = build_model(layer_dims=layer_dims, learning_rate=learning_rate, 
                            use_regularizer=True, regularizer=regularizer, 
                            initializer=initializer, activation=activation)
        f1_score = train(X_train_scaled, Y_train, X_dev_scaled, Y_dev, model, 
                        batch_size=batch_size)
        
        return f1_score
    return objective1

In [13]:
def objective2(trial):
    n1scale = trial.suggest_int('n1scale', 3, 10)
    n2scale = trial.suggest_int('n2scale', 3, 10)
    n3scale = trial.suggest_int('n3scale', 3, 10)
    n4scale = trial.suggest_int('n4scale', 3, 10)
    layer_dims = [INPUT_SIZE, int(2**n1scale), int(2**n2scale), int(2**n3scale), int(2**n4scale), OUTPUT_SIZE]

    print('Testing network shape', layer_dims)

    study = optuna.create_study(study_name='network_pickup', direction='maximize')
    study.optimize(create_objective_1(layer_dims), n_trials=10, show_progress_bar=False)

    trial = study.best_trial
    trial.set_user_attr('layer_dims', layer_dims)
    print(f'-> Best score: {trial.value}')
    print(f'-> Optimal hyperparameters: ')
    pprint.pprint(trial.params)

    return trial.value

In [14]:
from optuna.visualization import plot_optimization_history, plot_slice

study = optuna.create_study(study_name='dl8', direction='maximize')
study.optimize(objective2, n_jobs=2, n_trials=10, show_progress_bar=True)

trial = study.best_trial
print(f'-> Best score: {trial.value}')
print(f'-> Optimal network parameters: ')
pprint.pprint(trial.params)
print(f'-> Best learning parameters: ')
pprint.pprint(trial.user_attrs)


[I 2024-04-14 14:12:05,056] A new study created in memory with name: dl8
                                      

Testing network shape [2003, 64, 8, 32, 8, 1]
Testing network shape [2003, 1024, 256, 16, 512, 1]
[I 2024-04-14 14:12:05,059] A new study created in memory with name: network_pickup


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2024-04-14 14:12:05,060] A new study created in memory with name: network_pickup
24/24 [==============================] - 0s 2ms/step


  0%|          | 0/10 [00:15<?, ?it/s]

...

Validation F1 score: 0.006006006006006006
[I 2024-04-14 14:25:10,157] Trial 7 finished with value: 0.006006006006006006 and parameters: {'learning_rate': 2.7811349144791358e-05, 'batch_size': 6851, 'regularizer': 0.00024092832826331479, 'initializer': 'he_normal', 'activation': 'leaky_relu'}. Best is trial 3 with value: 0.7550077041602464.
24/24 [==============================] - 0s 779us/step


Best trial: 0. Best value: 0.763425:  90%|█████████ | 9/10 [13:07<01:20, 80.83s/it]

Validation F1 score: 0.005917159763313609
[I 2024-04-14 14:25:12,862] Trial 8 finished with value: 0.005917159763313609 and parameters: {'learning_rate': 3.6776945294704684e-05, 'batch_size': 3426, 'regularizer': 0.002864680892605136, 'initializer': 'he_normal', 'activation': 'gelu'}. Best is trial 3 with value: 0.7550077041602464.
24/24 [==============================] - 0s 806us/step


Best trial: 0. Best value: 0.763425: 100%|██████████| 10/10 [13:10<00:00, 79.03s/it]

Validation F1 score: 0.0
[I 2024-04-14 14:25:15,393] Trial 9 finished with value: 0.0 and parameters: {'learning_rate': 1.2654931377826447e-05, 'batch_size': 1713, 'regularizer': 0.0032079396563949045, 'initializer': 'glorot_normal', 'activation': 'relu'}. Best is trial 3 with value: 0.7550077041602464.
-> Best score: 0.7550077041602464
-> Optimal hyperparameters: 
{'activation': 'leaky_relu',
 'batch_size': 1713,
 'initializer': 'he_normal',
 'learning_rate': 0.0009880487825502965,
 'regularizer': 2.9024184634812676e-05}
[I 2024-04-14 14:25:15,395] Trial 9 finished with value: 0.7550077041602464 and parameters: {'n1scale': 3, 'n2scale': 3, 'n3scale': 4, 'n4scale': 3}. Best is trial 0 with value: 0.7634252539912917.
-> Best score: 0.7634252539912917
-> Optimal network parameters: 
{'n1scale': 6, 'n2scale': 3, 'n3scale': 5, 'n4scale': 3}
-> Best learning parameters: 
{}


In [15]:
nn = trial.params
nn_dims = [2**nn[f'n{l+1}scale'] for l in range(len(nn))]
nn_dims.insert(0, INPUT_SIZE)
nn_dims.append(OUTPUT_SIZE)
print('Optimal NN dimensions', nn_dims)

Optimal NN dimensions [2003, 64, 8, 32, 8, 1]


In [20]:
plot_optimization_history(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'mode': 'markers',
              'name': 'Objective Value',
              'type': 'scatter',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
              'y': [0.7634252539912917, 0.7611464968152866, 0.7573415765069552,
                    0.7578616352201258, 0.7500000000000001, 0.7492163009404389,
                    0.7608346709470305, 0.7606973058637084, 0.7531645569620252,
                    0.7550077041602464]},
             {'mode': 'lines',
              'name': 'Best Value',
              'type': 'scatter',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
              'y': [0.7634252539912917, 0.7634252539912917, 0.7634252539912917,
                    0.7634252539912917, 0.7634252539912917, 0.7634252539912917,
                    0.7634252539912917, 0.7634252539912917, 0.7634252539912917,
                    0.7634252539912917]},
             {'marker': {'color': '#cccccc'},
              'mode': 'markers',
              'name': 'Infeasible Trial',
              'showlegend': False,
              'type': 'scatter',
              'x': [],
              'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Optimization History Plot'},
               'xaxis': {'title': {'text': 'Trial'}},
               'yaxis': {'title': {'text': 'Objective Value'}}}
})

In [21]:
def objective_3(trial):
    n1scale = trial.suggest_int('n1scale', 3, 10)
    n2scale = trial.suggest_int('n2scale', 3, 10)
    n3scale = trial.suggest_int('n3scale', 3, 10)
    n4scale = trial.suggest_int('n4scale', 3, 10)
    layer_dims = [INPUT_SIZE, int(2**n1scale), int(2**n2scale), int(2**n3scale), int(2**n4scale), OUTPUT_SIZE]
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_categorical('batch_size', [BATCH_SIZE, BATCH_SIZE//2 + 1, BATCH_SIZE//4 + 1])
    regularizer = trial.suggest_float('regularizer', 1e-6, 1e-2, log=True)
    initializer = trial.suggest_categorical('initializer', ['glorot_normal', 'he_normal'])
    activation = trial.suggest_categorical('activation', ['relu', 'gelu', 'leaky_relu'])

    model = build_model(layer_dims=layer_dims, learning_rate=learning_rate, 
                        use_regularizer=True, regularizer=regularizer, 
                        initializer=initializer, activation=activation)
    f1_score = train(X_train_scaled, Y_train, X_dev_scaled, Y_dev, model, 
                    batch_size=batch_size)
    return f1_score


In [22]:
study_2 = optuna.create_study(study_name='dl8', direction='maximize')
study_2.optimize(objective_3, n_jobs=2, n_trials=30, show_progress_bar=True)

trial_2 = study_2.best_trial
print(f'-> Best score: {trial_2.value}')
print(f'-> Optimal network parameters: ')
pprint.pprint(trial_2.params)


[I 2024-04-14 14:55:51,149] A new study created in memory with name: dl8
  0%|          | 0/30 [00:00<?, ?it/s]

24/24 [==============================] - 0s 5ms/step


Best trial: 1. Best value: 0.747287:   3%|▎         | 1/30 [00:20<10:01, 20.73s/it]

Validation F1 score: 0.7472868217054264
[I 2024-04-14 14:56:11,872] Trial 1 finished with value: 0.7472868217054264 and parameters: {'n1scale': 7, 'n2scale': 8, 'n3scale': 9, 'n4scale': 3, 'learning_rate': 0.00019092486831908558, 'batch_size': 3426, 'regularizer': 0.00202148650457301, 'initializer': 'glorot_normal', 'activation': 'leaky_relu'}. Best is trial 1 with value: 0.7472868217054264.
24/24 [==============================] - 0s 3ms/step


Best trial: 1. Best value: 0.747287:   7%|▋         | 2/30 [00:24<04:59, 10.69s/it]

...

Validation F1 score: 0.7539936102236422
[I 2024-04-14 14:58:50,647] Trial 27 finished with value: 0.7539936102236422 and parameters: {'n1scale': 6, 'n2scale': 6, 'n3scale': 8, 'n4scale': 4, 'learning_rate': 0.00023593317450194392, 'batch_size': 3426, 'regularizer': 2.5178408676405703e-05, 'initializer': 'glorot_normal', 'activation': 'leaky_relu'}. Best is trial 15 with value: 0.7578616352201258.
24/24 [==============================] - 0s 2ms/step


Best trial: 15. Best value: 0.757862:  97%|█████████▋| 29/30 [03:03<00:03,  3.74s/it]

Validation F1 score: 0.7492063492063491
[I 2024-04-14 14:58:54,660] Trial 28 finished with value: 0.7492063492063491 and parameters: {'n1scale': 6, 'n2scale': 6, 'n3scale': 8, 'n4scale': 4, 'learning_rate': 0.00043743160082408597, 'batch_size': 3426, 'regularizer': 1.9604679386201076e-05, 'initializer': 'glorot_normal', 'activation': 'gelu'}. Best is trial 15 with value: 0.7578616352201258.
24/24 [==============================] - 0s 2ms/step


Best trial: 15. Best value: 0.757862: 100%|██████████| 30/30 [03:12<00:00,  6.42s/it]

Validation F1 score: 0.7380191693290735
[I 2024-04-14 14:59:03,716] Trial 29 finished with value: 0.7380191693290735 and parameters: {'n1scale': 9, 'n2scale': 3, 'n3scale': 4, 'n4scale': 9, 'learning_rate': 0.0004532934234041938, 'batch_size': 3426, 'regularizer': 0.00019815711008320502, 'initializer': 'glorot_normal', 'activation': 'gelu'}. Best is trial 15 with value: 0.7578616352201258.
-> Best score: 0.7578616352201258
-> Optimal network parameters: 
{'activation': 'gelu',
 'batch_size': 3426,
 'initializer': 'glorot_normal',
 'learning_rate': 0.00043572351744013946,
 'n1scale': 4,
 'n2scale': 5,
 'n3scale': 8,
 'n4scale': 6,
 'regularizer': 7.5959326184290255e-06}


In [25]:
plot_optimization_history(study_2)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'mode': 'markers',
              'name': 'Objective Value',
              'type': 'scatter',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
              'y': [0.335, 0.7472868217054264, 0.7377049180327869,
                    0.7569955817378498, 0.7560581583198709, 0.7477203647416413,
                    0.6303501945525292, 0.6860254083484574, 0.6611053180396247,
                    0.7363344051446946, 0.6932849364791288, 0.7547169811320756,
                    0.7519025875190258, 0.7523510971786833, 0.7562499999999999,
                    0.7578616352201258, 0.48322147651006714, 0.6328125,
                    0.7359735973597359, 0.6832740213523131, 0.7519747235387045,
                    0.6847826086956522, 0.7195945945945945, 0.7115044247787611,
                    0.7523510971786833, 0.7441130298273154, 0.5821205821205822,
                    0.7539936102236422, 0.7492063492063491, 0.7380191693290735]},
             {'mode': 'lines',
              'name': 'Best Value',
              'type': 'scatter',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
              'y': [0.335, 0.7472868217054264, 0.7472868217054264,
                    0.7569955817378498, 0.7569955817378498, 0.7569955817378498,
                    0.7569955817378498, 0.7569955817378498, 0.7569955817378498,
                    0.7569955817378498, 0.7569955817378498, 0.7569955817378498,
                    0.7569955817378498, 0.7569955817378498, 0.7569955817378498,
                    0.7578616352201258, 0.7578616352201258, 0.7578616352201258,
                    0.7578616352201258, 0.7578616352201258, 0.7578616352201258,
                    0.7578616352201258, 0.7578616352201258, 0.7578616352201258,
                    0.7578616352201258, 0.7578616352201258, 0.7578616352201258,
                    0.7578616352201258, 0.7578616352201258, 0.7578616352201258]},
             {'marker': {'color': '#cccccc'},
              'mode': 'markers',
              'name': 'Infeasible Trial',
              'showlegend': False,
              'type': 'scatter',
              'x': [],
              'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Optimization History Plot'},
               'xaxis': {'title': {'text': 'Trial'}},
               'yaxis': {'title': {'text': 'Objective Value'}}}
})